# New find_date_ranges

Resulta que el código de la función find_date_ranges está haciendo cosas extrañas. Parece que la implementación con merge_asof() funciona bien, pero vamos a hacer pruebas en este notebook para asegurarnos de que todo funciona correctamente.

To Do:
- Revisar el notebook a partir de sección 1.

## Creación de test dataset

Vamos a crear un dataset que tenga todas las variantes de datos que podamos encontrar

In [ ]:
import pandas as pd

input = []
visit = []

# 1 -> Person that has one event and one visit that match
input_rows = [{'person_id': 1, 'event_id': 1, 'start_date': '2020-01-05', 'expected_visit_id': 1}]
visit_rows = [{'person_id': 1, 'visit_id': 1, 'start_date': '2020-01-01', 'end_date': '2020-01-10',}]
input += input_rows
visit += visit_rows
# 2 -> Person that has one event and one visit that do not match
input_rows = [{'person_id': 2, 'event_id': 2, 'start_date': '2020-02-05', 'expected_visit_id': None}]
visit_rows = [{'person_id': 2, 'visit_id': 2, 'start_date': '2020-01-01', 'end_date': '2020-01-10'}]
input += input_rows
visit += visit_rows
# 3 -> Person that has one event and no visit
input_rows = [{'person_id': 3, 'event_id': 3, 'start_date': '2020-02-05', 'expected_visit_id': None}]
input += input_rows
# 4 -> Person that has no event and one visit 
visit_rows = [{'person_id': 4, 'visit_id': 3, 'start_date': '2020-01-01', 'end_date': '2020-01-10'}]
visit += visit_rows
# 5 -> Person that has two events and two visits. One match and the rest do not
input_rows = [{'person_id': 5, 'event_id': 4, 'start_date': '2020-01-05', 'expected_visit_id': 4}]
visit_rows = [{'person_id': 5, 'visit_id': 4, 'start_date': '2020-01-01', 'end_date': '2020-01-10'}]
input += input_rows
visit += visit_rows
input_rows = [{'person_id': 5, 'event_id': 5, 'start_date': '2020-02-05', 'expected_visit_id': None}]
visit_rows = [{'person_id': 5, 'visit_id': 5, 'start_date': '2020-03-01', 'end_date': '2020-03-10'}]
input += input_rows
visit += visit_rows
# 6 -> Person that has two events tha match to a single visit
input_rows = [{'person_id': 6, 'event_id': 6, 'start_date': '2020-01-04', 'expected_visit_id': 6},
              {'person_id': 6, 'event_id': 7, 'start_date': '2020-01-05', 'expected_visit_id': 6},]
visit_rows = [{'person_id': 6, 'visit_id': 6, 'start_date': '2020-01-01', 'end_date': '2020-01-10'}]
input += input_rows
visit += visit_rows
# 7 -> Person that has one event that fits the end of one period and the beginning of the next
input_rows = [{'person_id': 7, 'event_id': 8, 'start_date': '2020-01-05', 'expected_visit_id': 6}]
visit_rows = [{'person_id': 7, 'visit_id': 7, 'start_date': '2020-01-01', 'end_date': '2020-01-05'},
              {'person_id': 7, 'visit_id': 8, 'start_date': '2020-01-05', 'end_date': '2020-01-10'}]
input += input_rows
visit += visit_rows

In [ ]:
input_df = pd.DataFrame.from_records(input)
visit_df = pd.DataFrame.from_records(visit)
visit_df = visit_df.rename({
    'visit_id':'visit_occurrence_id',
    'start_date':'visit_start_date',
    'end_date':'visit_end_date',
}, axis=1)


In [ ]:
input_df

In [ ]:
visit_df

### Búsqueda de visit_occurrence_id
El siguiente paso consiste en enlazar cada medida del paciente con una visita. Para ello cargaremos la tabla VISIT_OCCURRENCE, que ya debería haber sido construida en una sección anterior, y buscaremos para cada measurement_date de la tabla MEASUREMENT un intervalo de fechas de visitas que la contenga. Si existe, le asignaremos el visit_occurrence_id correspondiente.

In [ ]:
def debug_find_date_ranges(input_df, visits_df, debug=False, method='index'):
    # Check for required columns in input_df
    required_input_columns = {'person_id', 'start_date'}
    missing_input_columns = required_input_columns - \
        set(input_df.columns)
    if missing_input_columns:
        raise ValueError(
            f"Missing required columns in input_df: {missing_input_columns}")

    # Check for required columns in visits_df
    required_visit_columns = [
        'person_id', 'visit_start_date', 'visit_end_date', 'visit_occurrence_id']
    missing_visit_columns = set(
        required_visit_columns) - set(visits_df.columns)
    if missing_visit_columns:
        raise ValueError(
            f"Missing required columns in visits_df: {missing_visit_columns}")

    # Merge the dataframes
    merged_df = pd.merge(
        input_df.reset_index(drop=True),
        visits_df[required_visit_columns].reset_index(drop=True),
        on='person_id',
        how='left'
    )

    # Check if merge resulted in any matches
    if merged_df['visit_occurrence_id'].isna().all():
        raise ValueError(
            ("No matching records found after merging."
             + "Check if person_id values align between dataframes."))

    # Create mask for dates within range
    date_range_mask = ((merged_df['start_date'] >= merged_df['visit_start_date']) &
                       (merged_df['start_date'] <= merged_df['visit_end_date']))

    # Filter only valid ranges
    valid_ranges = merged_df[date_range_mask]

    # Now we have two options
    # a) Use indexing to retrieve events without visit_id
    # b) Merge with the initial datatrame
    if method == 'index':
        # Get list of events that do have visit_id
        event_list_pos = set(valid_ranges['event_id'].unique())
        # Get list of events that do not have a visit_id
        event_list_neg = set(input_df['event_id'].values)-event_list_pos
        # Get subset of input with no id and put the extra columns
        leftover_df = input_df[input_df['event_id'].isin(event_list_neg)]
        leftover_df['visit_start_date'] = None
        leftover_df['visit_end_date'] = None
        leftover_df['visit_occurrence_id'] = None
        # Build the final df
        final_df = pd.concat([valid_ranges, leftover_df], axis=0)
    elif method == 'merge':
        final_df = pd.merge(
            input_df,
            valid_ranges[['person_id', 'event_id', 'visit_start_date',
                'visit_end_date', 'visit_occurrence_id']],
            on=['person_id','event_id'],
            how='left',
        )
    else:
        raise SyntaxError("method parameter is not correct. Use 'index' or 'method'")
    final_df = final_df.drop_duplicates(['person_id', 'event_id'])
    
    # Check if any valid ranges were found
    if final_df.empty:
        print(("Warning: No valid date ranges found."
              + "All condition start dates are outside visit date ranges."))
    if debug:
        # Check for duplicates in input_df
        input_duplicates = input_df.duplicated().sum()
        print(f"Number of duplicates in input_df: {input_duplicates}")

        # Check for duplicates in visits_df
        visits_duplicates = visits_df.duplicated().sum()
        print(f"Number of duplicates in visits_df: {visits_duplicates}")
        
        # Print shape of input dataframes
        print(f"Shape of input_df: {input_df.shape}")
        print(f"Shape of visits_df: {visits_df.shape}")        
        # Print shape of merged dataframe
        print(f"Shape of merged_df: {merged_df.shape}")
        # Print shape of valid_ranges
        print(f"Shape of valid_ranges: {valid_ranges.shape}")
        # Print shape of valid_ranges
        print(f"Shape of final_df: {final_df.shape}")
        
    return final_df


df = debug_find_date_ranges(input_df, visit_df, debug=True, method='merge')
df

Con la última implementación a somos capaces de recuperar todos los eventos, tengan visit_occurrence_id o no.

In [ ]:
def asof_find_date_ranges(input_df, visits_df, debug=False):
    pd.options.mode.copy_on_write = True
    # Check for required columns in input_df
    required_input_columns = {'person_id', 'start_date'}
    missing_input_columns = required_input_columns - \
        set(input_df.columns)
    if missing_input_columns:
        raise ValueError(
            f"Missing required columns in input_df: {missing_input_columns}")

    # Check for required columns in visits_df
    required_visit_columns = [
        'person_id', 'visit_start_date', 'visit_end_date', 'visit_occurrence_id']
    missing_visit_columns = set(required_visit_columns) - set(visits_df.columns)
    if missing_visit_columns:
        raise ValueError(
            f"Missing required columns in visits_df: {missing_visit_columns}")
    if debug:
        print(f"Shape of input_df: {input_df.shape}")
        print(f"Shape of visits_df: {visits_df.shape}")

    # Ensure start_date and visit dates are datetime
    input_df['start_date'] = pd.to_datetime(input_df['start_date'])
    visits_df['visit_start_date'] = pd.to_datetime(visits_df['visit_start_date'])
    visits_df['visit_end_date'] = pd.to_datetime(visits_df['visit_end_date'])

    # Sort the dataframes
    input_df = input_df.sort_values(['person_id', 'start_date'])
    visits_df = visits_df.sort_values(['person_id', 'visit_start_date', 'visit_end_date'])

    def merge_for_person(person_data, person_visits):
        merged = pd.merge_asof(
            person_data,
            person_visits,
            left_on='start_date',
            right_on='visit_start_date',
            direction='backward'
        )
        return merged

    # Group by person_id and apply merge_asof for each person
    grouped_input = input_df.groupby('person_id')
    grouped_visits = visits_df.groupby('person_id')

    result_dfs = []
    for person_id, person_data in grouped_input:
        person_visits = grouped_visits.get_group(person_id) if person_id in grouped_visits.groups else pd.DataFrame()
        if not person_visits.empty:
            result_dfs.append(merge_for_person(person_data, person_visits))

    # Combine results
    if result_dfs:
        merged_df = pd.concat(result_dfs, ignore_index=True)
    else:
        merged_df = pd.DataFrame()

    # Filter for valid ranges
    valid_ranges = merged_df[
        (merged_df['start_date'] >= merged_df['visit_start_date']) &
        (merged_df['start_date'] <= merged_df['visit_end_date'])
    ]

    if debug:
        print(f"Shape of valid_ranges: {valid_ranges.shape}")

    if valid_ranges.empty:
        print("Warning: No valid date ranges found. All condition start dates are outside visit date ranges.")

    return valid_ranges

df = asof_find_date_ranges(input_df,visit_df,debug=True)
df

Este igual, me borra todo lo que no tenga un match

In [ ]:
def brute_find_date_ranges(input_df, visits_df, debug=False):
    # == Preprocessing ============================================================
    # Check for required columns in input_df
    required_input_columns = {'person_id', 'start_date'}
    missing_input_columns = required_input_columns - \
        set(input_df.columns)
    if missing_input_columns:
        raise ValueError(
            f"Missing required columns in input_df: {missing_input_columns}")

    # Check for required columns in visits_df
    required_visit_columns = {
        'person_id', 'visit_start_date', 'visit_end_date', 'visit_occurrence_id'}
    missing_visit_columns = required_visit_columns - set(visits_df.columns)
    if missing_visit_columns:
        raise ValueError(
            f"Missing required columns in visits_df: {missing_visit_columns}")

    output_df = input_df.copy()
    output_df['visit_occurrence_id'] = None
    for input_idx, input_row in output_df.iterrows():
        # if debug:
        #     print(input_idx)
        tmp_visit = visits_df[visits_df['person_id'] == input_row['person_id']]
        if tmp_visit.shape[0] == 0:
            continue
        for visit_idx, visit_row in tmp_visit.iterrows():
            # if debug:
            #     print(' ', visit_idx)
            flag_start = input_row['start_date'] >= visit_row['visit_start_date']
            flag_end = input_row['start_date'] <= visit_row['visit_end_date']
            flag = flag_start & flag_end
            if flag:
                output_df.loc[input_idx, 'visit_occurrence_id'] = visit_row['visit_occurrence_id']

    if debug:
        print(f"Shape of input_df: {input_df.shape}")
        print(f"Shape of visits_df: {visits_df.shape}")
        print(f"Shape of output_df: {output_df.shape}")

    return output_df


df = brute_find_date_ranges(input_df, visit_df, debug=True)
df

In [ ]:
import sys
sys.path.append('..')
import bps_to_omop.general as gen

df = gen.find_date_ranges(input_df[['person_id','start_date','event_id']],
                          visit_df, verbose=2)
df

In [ ]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)    
    print('debug_find_date_ranges index')
    %timeit -n 5 -r 5 debug_find_date_ranges(input_df.copy(),visit_df.copy(), method='index')
    print('debug_find_date_ranges merge')
    %timeit -n 5 -r 5 debug_find_date_ranges(input_df.copy(),visit_df.copy(), method='merge')
    print('gen.find_date_ranges (eq to debug_*_mege)')
    %timeit -n 5 -r 5 gen.find_date_ranges(input_df.copy(),visit_df.copy(), verbose=0)
    print('asof_find_date_ranges')
    %timeit -n 5 -r 5 asof_find_date_ranges(input_df.copy(),visit_df.copy())
    print('brute_find_date_ranges')
    %timeit -n 5 -r 5 brute_find_date_ranges(input_df.copy(),visit_df.copy())

## Prueba con datasets grandes
Vamos a comparar si el metodo de pyarrow sigue funcionando más rápido con datasets grandes.

Nos traemos la función para generar datasets

In [ ]:
import numpy as np
import pandas as pd
import pyarrow as pa
from pyarrow import parquet

import sys
sys.path.append('..')
import bps_to_omop.general as gen

def create_sample_df(n_people: int = 1000, 
                     n_dates: int = 50,
                     first_date: str = '2020-01-01',
                     last_date: str = '2023-01-01',
                     mean_duration_days: int = 60,
                     std_duration_days: int = 180,
                     people_pool = None) -> pd.DataFrame:
    """
    Creates a dataframe of 'n_people' people with 'n_dates' events each.
    
    Events will be restrained to start after 'first_date' and do not
    begin after 'last_date'.
    
    The events and their duration will be modelled with a gaussian with
    mean = 'mean_duration_days' and std='std_duration_days'.
    
    User can provide a list of person_id using people_pool. If provided
    no new users will be created, but 'n_dates' events will be drawn
    from each 'person_id' in 'people_pool' that list of ids.
    """
    # == Parameters ==
    np.random.seed(42)
    pd.options.mode.string_storage = "pyarrow"
    # Start date from which to start the dates
    first_date = pd.to_datetime(first_date)
    last_date = pd.to_datetime(last_date)
    max_days = (last_date-first_date).days

    # == Generate IDs randomly ==
    # -- Generate the Ids
    if people_pool is None:
        size = n_people*n_dates
        people = np.random.randint(10000000, 99999999 + 1, size=n_people)
        person_id = np.random.choice(people, size)
    else:
        size = n_dates
        person_id = np.random.choice(people_pool, size)

    # == Generate random dates ==
    # Generate random integers for days and convert to timedelta
    random_days = np.random.randint(0, max_days, size=size)
    # Create the columns
    observation_start_date = first_date + \
        pd.to_timedelta(random_days, unit='D')
    # Generate a gaussian sample of dates
    random_days = np.random.normal(
        mean_duration_days, std_duration_days, size=size)
    random_days = np.int32(random_days)
    observation_end_date = observation_start_date + \
        pd.to_timedelta(random_days, unit='D')
    # Correct end_dates
    # => If they are smaller than start_date, take start_date
    observation_end_date = np.where(observation_end_date < observation_start_date,
                                    observation_start_date, observation_end_date)

    # == Generate the code ==
    event_id = np.arange(len(person_id))

    # == Generate the dataframe ==
    df_raw = {'event_id': event_id, 
              'person_id': person_id, 
              'start_date': observation_start_date,
              'end_date': observation_end_date, }
    df_raw = pd.DataFrame(df_raw)
    df_raw['start_date'] = pd.to_datetime(df_raw['start_date']).astype('datetime64[ms]')
    df_raw['end_date'] = pd.to_datetime(df_raw['end_date']).astype('datetime64[ms]')
    
    return df_raw

Probamos con datos pequeños que se puedan manejar

In [ ]:
# Define parameters
n_people = 2
n_dates = 10
last_date = '2020-07-31'
# Create visit dataset
visit_df = create_sample_df(n_people=n_people,n_dates=n_dates,last_date=last_date)
visit_df = visit_df.sort_values(['person_id','start_date','end_date'])
visit_df = visit_df.rename({
    'event_id':'visit_occurrence_id',
    'start_date':'visit_start_date',
    'end_date':'visit_end_date',
}, axis=1)
visit_df = gen.remove_overlap(visit_df)
visit_df


In [ ]:
n_dates = 4
# Create input dataset
input_df = create_sample_df(n_dates=n_dates,
                            last_date=last_date,
                            people_pool=visit_df['person_id'].unique())
input_df = input_df.drop('end_date',axis=1).sort_values(['person_id','start_date'])
input_df

In [ ]:
df = brute_find_date_ranges(input_df, visit_df)
df


Aquí vemos que el brute find_date_ranges encuentra todo lo que tiene que encontrar. Todas las citas del person_id 66755036 tienen una cita asociada. Mientras que la otra person_id, 75682867, no tiene ninguna visita que incluya las fechas de su único evento.

In [ ]:
import warnings

# Define parameters
n_people = 30000
n_dates_visit = 100
n_dates_input = 500000
last_date = '2020-12-31'

# Create visit dataset
print('Creating visit dataset...')
visit_df = create_sample_df(
    n_people=n_people, n_dates=n_dates_visit, last_date=last_date)
visit_df = visit_df.rename({
    'event_id': 'visit_occurrence_id',
    'start_date': 'visit_start_date',
    'end_date': 'visit_end_date',
}, axis=1)
visit_df = gen.remove_overlap(visit_df, verbose=1)
visit_df = visit_df[['person_id', 'visit_start_date',
                     'visit_end_date', 'visit_occurrence_id']]
visit_df = gen.group_dates(visit_df, n_days=60, verbose=1)
visit_df['visit_occurrence_id'] = np.arange(len(visit_df))

# Create input dataset
print('Creating input dataset...')
input_df = create_sample_df(n_dates=n_dates_input,
                            last_date=last_date,
                            people_pool=visit_df['person_id'].unique())
input_df = input_df.drop('end_date', axis=1)

Primero comprobamos que los resultados obtenidos tienen sentido

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)    
    print('\ndebug_find_date_ranges index')
    debug_find_date_ranges(input_df,visit_df, debug=True, method='index')
    print('\ndebug_find_date_ranges merge')
    debug_find_date_ranges(input_df,visit_df, debug=True, method='merge')
    print('\ngen.find_date_ranges (eq to debug_*_mege)')
    gen.find_date_ranges(input_df,visit_df, verbose=2)
    # print('\nasof_find_date_ranges')
    # asof_find_date_ranges(input_df,visit_df, debug=True)
    # print('\nbrute_find_date_ranges')
    # brute_find_date_ranges(input_df,visit_df, debug=True)

Luego comprobamos la velocidad

In [ ]:
with warnings.catch_warnings():
    print(f"{n_people = }\n{n_dates_visit = }\n{n_dates_input = }")
    warnings.simplefilter(action='ignore', category=FutureWarning)    
    print('\ndebug_find_date_ranges index')
    %timeit -n 5 -r 5 debug_find_date_ranges(input_df,visit_df, method='index')
    print('\ndebug_find_date_ranges merge')
    %timeit -n 5 -r 5 debug_find_date_ranges(input_df,visit_df, method='merge')
    print('\ngen.find_date_ranges (eq to debug_*_mege)')
    %timeit -n 5 -r 5 gen.find_date_ranges(input_df,visit_df, verbose=0)
    # print('\nasof_find_date_ranges')
    # %timeit -n 5 -r 5 asof_find_date_ranges(input_df,visit_df)
    # print('\nbrute_find_date_ranges')
    # %timeit -n 5 -r 5 brute_find_date_ranges(input_df,visit_df)

No se nota la diferencia entre el método por fuerza bruta y los otros hasta que no aumenta bastante el tamaño de los dataframes (>1000)

El método asof no está finalizado, ya que sólo devuelve eventos con visit_occurrence_id. Aún así es más lento, ya que sigue teniendo un bucle.

Los primeros tres métodos (debug-index, debug-merge y el implementado en gen) son muy similares, con tiempos similares incluso para grandes volúmenes de datos.

Ordenar o no el dataframe dentro de la función find_date_range() parece tener un pequeño efecto en la velocidad, siendo levemente más rápido (190 ms a 170 ms). Esto parece ser relevante sólo para dataframes grandes (>10000)

In [ ]:
input_df = input_df.sort_values(['person_id','event_id','start_date'])
input_df

In [ ]:
output_df = debug_find_date_ranges(input_df,visit_df, debug=True, method='index')
output_df = output_df.sort_values(['person_id','event_id','start_date'])
output_df